# Filtering

Event filtering refers to the process of removing or extracting a subset of events based on some criterion such as the temperature of the measured sample at the time an event was detected.
Scipp's binned data can be used for this purpose.

Below, we describe two cases.
In the simple case the data contains the required coordinate and [scipp.bin](../../generated/functions/scipp.bin.rst) can be used directly.
In the more complex case metadata requires preprocessing, and generally there are three steps to take:

1. Preprocess the metadata used for filtering.
   For example, a noisy time series of temperature values needs to converted into a series of time intervals with a fixed temperature value within the interval.
   This process might involve defining thresholds and tolerances or interpolation methods between measured temperature values.
2. Map event timestamps to temperature values.
3. Filter data based on temperature values.

## Preparation

We create some fake data for illustration purposes.

<div class="alert alert-info">

**Note**

In practice data to be filtered would be based on a loaded file. Details of this subsection can safely by skipped, as long as all cells are executed.

</div>

In [ ]:
import numpy as np
import scipp as sc

In [ ]:
np.random.seed(1) # Fixed for reproducibility
end_time = 100000
tof_max = 10000
width = tof_max/20
sizes = 4*np.array([7000, 3333, 3000, 5000])
size = np.sum(sizes)
data = sc.ones(dims=['event'], unit='counts', shape=[size], with_variances=True)
time = sc.zeros(dims=['event'], unit='s', dtype='datetime64', shape=[size])
# time-of-flight in a neutron-scattering experiment
tof = sc.zeros(dims=['event'], unit='us', dtype='float64', shape=[size])
table = sc.DataArray(data=data, coords={'time':time, 'tof':tof})
table

In [ ]:
ntemp = 100
sample_temperature = sc.DataArray(
    data=sc.array(dims=['time'], unit='K',
                  values=5*np.random.rand(100)+np.linspace(100, 120, num=ntemp)),
    coords={'time':sc.Variable(dims=['time'], unit='s',
                               values=np.linspace(0, end_time, num=ntemp).astype('datetime64[s]'))})
x = sc.linspace(dim='x', unit='m', start=0, stop=1, num=4)

end = sc.array(dims=['x'], values=np.cumsum(sizes), unit=None)
begin = end.copy()
begin.values -= sizes
events = sc.DataArray(
    data=sc.bins(begin=begin, end=end, dim='event', data=table),
    coords={'x': x},
    attrs={'sample_temperature': sc.scalar(value=sample_temperature)})
for size, bucket in zip(sizes, events.values):
    bucket.coords['time'].values = np.linspace(0, end_time, num=size).astype('datetime64[s]')
    bucket.coords['tof'].values = np.concatenate(
        (np.concatenate(
            (7*width + width*np.random.randn(size//4),
             13*width + width*np.random.randn(size//4))),
         10*width + width*np.random.randn(size//2)))
events

## Filtering based on existing coords

### Extracting data based on an interval

We can use [scipp.bin](../../generated/functions/scipp.bin.rst) with the desired bounds to extract all data points (events) that have coord values falling within an interval:

In [ ]:
tof_interval = sc.array(dims=['tof'], values=[2000.0, 3000.0], unit='us')
filtered = events.bin(tof=tof_interval)
filtered

### Extracting/splitting data based on multiple intervals

In the same manner, we can extract data with a list of (adjacent) intervals:

In [ ]:
tof_intervals = sc.linspace(dim='tof', start=2000, stop=3000, num=4, unit='us')
filtered = events.bin(tof=tof_intervals)
filtered

Events in each of the subintervals can then be accessed using the usual slicing syntax:

In [ ]:
filtered['tof',2]

## Filtering based on arbitrary metadata
### Step 1: Preprocess metadata

Our data contains a coordinate with metadata related to the temperature of the measured sample:

In [ ]:
timeseries = events.attrs['sample_temperature'].value
timeseries.plot()

This is a timeseries with noisy measurements, as could be obtained, e.g., from a temperature sensor.
For event filtering we require intervals with a fixed temperature.
This can be obtained in many ways.
In this example we do so by taking the mean over subintervals:

In [ ]:
average=4
temperature = sc.fold(timeseries, dim='time', sizes={'time': ntemp//average, 'dummy': average})
time_coord = temperature.coords['time']['dummy', 0]
temperature.coords['time'] = sc.concat([time_coord, time_coord['time', -1] + 1*sc.units.s], 'time')
temperature = temperature.mean('dummy')
temperature.plot()

### Step 2: Map time stamps

The `temperature` data array computed above can be seen as a discretized functional dependence of temperature on time.
This "function" can now be used to map the `time` of each event to the `temperature` of each event:

In [ ]:
event_temp = sc.lookup(temperature, 'time')[events.bins.coords['time']]
events.bins.coords['temperature'] = event_temp

The event lists with temperature values created by `scipp.map` have been added as a new coordinate:

In [ ]:
events.values[0]

### Step 3: Filter with `scipp.bin`

The temperature coordinate created in the previous step can now be used for the actual filtering step.
With a `temperature` coordinate stored as part of `events` it is possible to use `scipp.bin` with temperature bins:

In [ ]:
temp_bins = sc.linspace('temperature', 100.0, 130.0, num=6, unit='K')
binned_events = events.bin(temperature=temp_bins, tof=100)
binned_events

Filtering is then performed by slicing and, if desired, copying:

In [ ]:
filtered_view = binned_events['temperature', 0:3] # view containing only relevant events
filtered = binned_events['temperature', 0:3].copy() # extract only relevant events by copying

Slicing combined with histogramming also performs a filter operation since all events outside the histogram bounds are dropped:

In [ ]:
binned_events['temperature', 1].hist().plot()

In [ ]:
binned_events['temperature', 3].hist().plot()

Results from filter operations can also be inserted into a dataset for convenient handling of further operations such as histogramming, summing, or plotting:

In [ ]:
d = sc.Dataset()
d['below_T_c'] = binned_events['temperature', 1]
d['above_T_c'] = binned_events['temperature', 3]
d.bins.sum().sum('x').plot()

We can also bin without the time-of-flight coordinate to obtain the temperature dependence of the total event count, e.g., for normalization purposes:

In [ ]:
binned_events = events.bin(temperature=temp_bins)
binned_events.hist(temperature=50).plot()